In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import pymorphy2

### Data preprocessing

In [3]:
seed = 42
positive_file = "../data/positive.csv"
negative_file = "../data/negative.csv"

In [4]:
np.random.seed(seed)

База данных состоит из 12 столбцов:

1. id: уникальный номер сообщения в системе twitter;
2. tdate: дата публикации сообщения (твита);
3. tmane: имя пользователя, опубликовавшего сообщение;
4. ttext:  текст сообщения (твита);
5. ttype: поле в котором в дальнейшем будет указано к кому классу относится твит (положительный, отрицательный, нейтральный);
6. trep: количество реплаев к данному сообщению. В настоящий момент API твиттера не отдает эту информацию;
7. trtw: число ретвитов
8. tfav: число сколько раз данное сообщение было добавлено в избранное другими пользователями;
9. tstcount: число всех сообщений пользователя в сети twitter;
10. tfoll: количество фоловеров пользователя (тех людей, которые читают пользователя);
11. tfrien: количество друзей пользователя (те люди, которых читает пользователь);
12. listcount: количество листов-подписок в которые добавлен твиттер-пользователь.

In [5]:
column_names = ["id", "tdate", "tmane", "ttext", "ttype", "trep", "trtw", "tfav", "tstcount", "tfoll", "tfrien", "listcount"]
positive_df = pd.read_csv(positive_file, sep=";", names=column_names, index_col=False)
negative_df = pd.read_csv(negative_file, sep=";", names=column_names, index_col=False)

In [6]:
positive_df.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [7]:
# Смена метки класса для отрицательной эмоциональной окраски
negative_df["ttype"] = 0

In [8]:
df = pd.concat([negative_df, positive_df])
df.shape, negative_df.shape, positive_df.shape

((226834, 12), (111923, 12), (114911, 12))

In [9]:
df = df[["ttext", "ttype"]]
df.columns = ['text', 'target']

In [10]:
df["text"].tolist()[:40]

['на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так D:',
 'Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(',
 '@elina_4post как говорят обещаного три года ждут...((',
 'Желаю хорошего полёта и удачной посадки,я буду очень сильно скучать( http://t.co/jCLNzVNv3S',
 'Обновил за каким-то лешим surf, теперь не работает простоплеер :(',
 'Котёнка вчера носик разбила, плакала и расстраивалась :(',
 '@juliamayko @O_nika55 @and_Possum Зашли, а то он опять затихарился, я прямо физически страдаю, когда он долго молчит!(((',
 'а вообще я не болею -  я не выздоравливаю :(',
 'я микрофраза :( учимся срать кирпичами в режиме &amp;quot;нон-стоп&amp;quot; @niwoqisipapy',
 'я хочу с тобой помириться , но сука я гордая и никогда этого не сделаю! (((',
 '@DNO_OKEANA_A3A3 @MOE_MOPE_A3A3 тебя ебет какие у меня фотки.я про твои молчу.и вообще ты хоть знаешь как ТП то выглядят...',
 'Блин начали сниться сны( Не когда почти не снились ! А теперь можно ска

### Стадии очистки текста:
1. Приведение к нижниму регистру
2. Замена буквы "ё" на "е"
3. Удаление цифр
4. Удаление HTML специальных символов
5. Замена упоминаний пользователей @username на тег at_user
6. Замена символа хештега # на тег hash
7. Удаление RT
8. Замена гиперссылок на тег url



В работе "Sentiment Analysis of Posts and Comments in the Accounts of Russian Politicians on the Social Network"
в разделе "III. DATA AND METHODOLOGY" рассматривается стадия распознования смайликов в последовательностях пунктуации 
и замену их на теги соответствующих смайлов. Это уменьшит "шум", создоваемый черезмерным употреблением символов пунктуации

Для простоты, я не стал удалять знаки пунктуации.

In [11]:
def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    # Remove digits
    text = re.sub("\d+:\d+", " ", text)
    text = re.sub(" \d+", " ", text)
    # Removing ;quot; and &amp
    text = re.sub(';quot;', ' ', text) 
    text = re.sub('&amp', ' ', text) 
    # Remove HTML special entities 
    text = re.sub(r'\&\w*;', ' ', text)
    #Convert @username to AT_USER
    text = re.sub('@[^\s]+','at_user', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+', ' ', text)
    # Removing '#' hash tag
    text = re.sub('#', 'hash ', text) 
    # Removing RT
    text = re.sub('rt[\s]+', '', text) 
    # Removing hyperlink
    text = re.sub('https?:\/\/\S+', 'url', text)
    # Separate words and punctuation
    text = re.findall(r"[\w']+|[.,!?;:()]", text)
    text = " ".join(text)
    return text

In [12]:
# Clean the tweets
df['text'] = df['text'].apply(preprocess_text)

In [13]:
df["text"].tolist()[:40]

['на работе был полный пиддес : и так каждое закрытие месяца , я же свихнусь так d :',
 'коллеги сидят рубятся в urban terror , а я из за долбанной винды не могу : (',
 'at_user как говорят обещаного три года ждут . . . ( (',
 'желаю хорошего полета и удачной посадки , я буду очень сильно скучать ( url',
 'обновил за каким то лешим surf , теперь не работает простоплеер : (',
 'котенка вчера носик разбила , плакала и расстраивалась : (',
 'at_user at_user at_user зашли , а то он опять затихарился , я прямо физически страдаю , когда он долго молчит ! ( ( (',
 'а вообще я не болею я не выздоравливаю : (',
 'я микрофраза : ( учимся срать кирпичами в режиме нон стоп at_user',
 'я хочу с тобой помириться , но сука я гордая и никогда этого не сделаю ! ( ( (',
 'at_user at_user тебя ебет какие у меня фотки . я про твои молчу . и вообще ты хоть знаешь как тп то выглядят . . .',
 'блин начали сниться сны ( не когда почти не снились ! а теперь можно сказать каждый день такое ! мне это не нравитьс

### Лемматизация

Использую лемматизатор от pymorphy2. Он не снимает омонимию, так как обрабатывате каждое сло по отдельности.
Для обычных текстов я бы использовал pymystem3. Он обрабатывает предложения целиком и способен различить разные слова с одинаковым написанием.

In [14]:
cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = []
    for token in text.split():
        # Если токен уже был закеширован, быстро возьмем результат из кэша.
        if token in cache.keys():
            words.append(cache[token])
        # Слово еще не встретилось, будем проводить медленный морфологический анализ.
        else:
            result = morph.parse(token)   
            word = result[0].normal_form
            # Отправляем слово в результат, ...
            words.append(word)
            # ... и кешируем результат его разбора.
            cache[token] = word   
    return ' '.join(words)

In [15]:
%%time

df['text'] = df['text'].apply(lemmatize)

CPU times: user 40.6 s, sys: 0 ns, total: 40.6 s
Wall time: 40.6 s


In [16]:
df["text"].tolist()[:40]

['на работа быть полный пиддес : и так каждый закрытие месяц , я же свихнуться так d :',
 'коллега сидеть рубиться в urban terror , а я из за долбать винд не мочь : (',
 'at_user как говорят обещаной три год ждать . . . ( (',
 'желать хороший полёт и удачный посадка , я быть очень сильно скучать ( url',
 'обновить за какой то леший surf , теперь не работать простоплеер : (',
 'котёнок вчера носик разбить , плакать и расстраиваться : (',
 'at_user at_user at_user заслать , а то он опять затихариться , я прямо физически страдать , когда он долго молчать ! ( ( (',
 'а вообще я не болеть я не выздоравливать : (',
 'я микрофраза : ( учиться срать кирпич в режим нона стоп at_user',
 'я хотеть с ты помириться , но сук я гордый и никогда это не сделать ! ( ( (',
 'at_user at_user ты ебета какой у я фотка . я про твой молчать . и вообще ты хоть знаешь как тп то выглядеть . . .',
 'блин начать сниться сон ( не когда почти не сниться ! а теперь можно сказать каждый день такой ! я это не нравиться

In [17]:
df = df.drop(df[df['text'].map(str) == 'nan'].index)

train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train.to_csv("../data/train_processed_data.csv", index=False)
validate.to_csv("../data/validate_processed_data.csv", index=False)
test.to_csv("../data/test_processed_data.csv", index=False)

train.shape, validate.shape, test.shape

((136100, 2), (45367, 2), (45367, 2))

In [18]:
train.head()

,text,target
103158,оставаться самый нужный и самый близкие ) весь...,1
108831,"такой приятный чувство , когда ты знаешь , что...",1
60463,день начинаться с лень вообще ничто делать не ...,0
101340,at_user at_user ксюша поход вплотную там суп з...,1
1132,"at_user с днём рождение at_user , творческий у...",1


In [19]:
validate.head()

,text,target
52653,at_user at_user я старушка ( ( (,0
1339,at_user at_user the king . url надо обожать ко...,0
48292,"ничто не разочаровывать сильный , чем дрянный ...",0
86267,hash happyvirthday вау классный фото ) ) url,1
47623,at_user повезти ! а у я и вклад там быть . и р...,0


In [20]:
test.head()

,text,target
16114,"at_user привееть , хелена : ) ) ) мимими . . ....",1
26645,"встать в , мама разбудить слово : ляля , встав...",1
6443,"что там на версуса быть , есть инф ? поход жес...",1
17759,"дима пригласить поиграть футбол , сейчас быть ...",1
73778,такой несчастный : ( микрорайон садовый url,0
